In [1]:
import findspark
findspark.init()

In [2]:
#Importing the packages
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as func

In [3]:
#Creating the SparkSession
spark = SparkSession.builder.appName("FirstApp").getOrCreate()

In [4]:
#Defining schema for your DataFrame
myschema = StructType([\
                       StructField("userID", IntegerType(), True),
                       StructField("name", StringType(), True),
                       StructField("age",IntegerType(), True),
                       StructField("friends",IntegerType(), True),
                        ])

In [5]:
#Creating DataFrame on a CSV file
people = spark.read.format("csv")\
    .schema(myschema)\
    .option("path","fakefriends.csv")\
    .load()

In [6]:
#Performing all thetransformations
output = people.select(people.userID,people.name\
                       ,people.age,people.friends)\
         .where(people.age < 30).withColumn('insert_ts', func.current_timestamp())\
         .orderBy(people.userID)

In [7]:
#taking the count of o/p DataFrame
output.count()

112

In [8]:
#create a temp view
output.createOrReplaceTempView('people')

In [9]:
#running a simple spark spl query
spark.sql('select name,age,friends,insert_ts from people').show()

+--------+---+-------+--------------------+
|    name|age|friends|           insert_ts|
+--------+---+-------+--------------------+
|Jean-Luc| 26|      2|2024-04-24 14:24:...|
|    Hugh| 27|    181|2024-04-24 14:24:...|
|  Weyoun| 22|    323|2024-04-24 14:24:...|
|   Miles| 19|    268|2024-04-24 14:24:...|
|  Julian| 25|      1|2024-04-24 14:24:...|
|     Ben| 21|    445|2024-04-24 14:24:...|
|  Julian| 22|    100|2024-04-24 14:24:...|
|     Nog| 26|    281|2024-04-24 14:24:...|
| Beverly| 27|    305|2024-04-24 14:24:...|
|    Morn| 25|     96|2024-04-24 14:24:...|
|   Brunt| 24|     49|2024-04-24 14:24:...|
|     Nog| 20|      1|2024-04-24 14:24:...|
| Beverly| 19|    269|2024-04-24 14:24:...|
|   Brunt| 19|      5|2024-04-24 14:24:...|
|  Geordi| 20|    100|2024-04-24 14:24:...|
|  Geordi| 21|    477|2024-04-24 14:24:...|
|  Kasidy| 22|    179|2024-04-24 14:24:...|
|   Brunt| 20|    384|2024-04-24 14:24:...|
|     Ben| 28|    311|2024-04-24 14:24:...|
|    Worf| 24|    492|2024-04-24

In [14]:
output.write \
    .format("parquet") \
    .option("path", "/home/christine/Desktop/spar/ouput") \
    .partitionBy("age") \
    .mode("append") \
    .save()
